In [3]:
# %load_ext autoreload
# %autoreload 2

import matplotlib
import matplotlib.style as mplstyle
matplotlib.use('TkAGG')
mplstyle.use('fast')

matplotlib.rcParams['path.simplify'] = True
matplotlib.rcParams['path.simplify_threshold'] = 1.0
matplotlib.rcParams['agg.path.chunksize'] = 10000

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.gridspec import GridSpec,GridSpecFromSubplotSpec
from matplotlib import animation, rc
from scipy.stats import f
from scipy.signal import find_peaks
from sklearn.cluster import MeanShift, estimate_bandwidth
from astropy.io import fits
from astropy.timeseries import LombScargle

from ResearchTools import LCtools

from astropy.table import Table
from astropy.coordinates import ICRS
import astropy.units as u
from astropy.coordinates import SkyCoord

import warnings
import tqdm.notebook as tqdm
from subprocess import *
import os
import pickle
import lzma
import bz2
import gzip
import blosc

from multiprocessing import Pool


In [27]:
output_dir = "Analysis_Results/"

datestr = check_output(["/bin/date","+%F"])
datestr = datestr.decode().replace('\n', '')
# datestr = '2021-06-15'
if not os.path.exists(output_dir+datestr):
    os.mkdir(output_dir+datestr)

lc_dir0 = output_dir+datestr+"/ZTF/"
lc_dir_CSS = output_dir+datestr+"/CSS/"
lc_dir_ZTFg = output_dir+datestr+"/ZTF/g"
lc_dir_ZTFr = output_dir+datestr+"/ZTF/r"
lc_dir_ZTFi = output_dir+datestr+"/ZTF/i"

if not os.path.exists(lc_dir0):
    os.mkdir(lc_dir0)
if not os.path.exists(lc_dir_CSS):
    os.mkdir(lc_dir_CSS)
if not os.path.exists(lc_dir_ZTFg):
    os.mkdir(lc_dir_ZTFg)
if not os.path.exists(lc_dir_ZTFr):
    os.mkdir(lc_dir_ZTFr)
if not os.path.exists(lc_dir_ZTFi):
    os.mkdir(lc_dir_ZTFi)
    
if not os.path.exists(output_dir+datestr+"/RAW_LC_ANALYSIS/"):
    os.mkdir(output_dir+datestr+"/RAW_LC_ANALYSIS/")

raw_lc_analysis_dir_ZTF = output_dir+datestr+"/RAW_LC_ANALYSIS/"+"/ZTF/"
raw_LC_analysis_dir_CSS = output_dir+datestr+"/RAW_LC_ANALYSIS/"+"/CSS/"
raw_LC_analysis_dir_ZTFg = output_dir+datestr+"/RAW_LC_ANALYSIS/"+"/ZTF/g/"
raw_LC_analysis_dir_ZTFr = output_dir+datestr+"/RAW_LC_ANALYSIS/"+"/ZTF/r/"
raw_LC_analysis_dir_ZTFi = output_dir+datestr+"/RAW_LC_ANALYSIS/"+"/ZTF/i/"   
    
if not os.path.exists(raw_lc_analysis_dir_ZTF):
    os.mkdir(raw_lc_analysis_dir_ZTF)
if not os.path.exists(raw_LC_analysis_dir_CSS):
    os.mkdir(raw_LC_analysis_dir_CSS)
if not os.path.exists(raw_LC_analysis_dir_ZTFg):
    os.mkdir(raw_LC_analysis_dir_ZTFg)
if not os.path.exists(raw_LC_analysis_dir_ZTFr):
    os.mkdir(raw_LC_analysis_dir_ZTFr)
if not os.path.exists(raw_LC_analysis_dir_ZTFi):
    os.mkdir(raw_LC_analysis_dir_ZTFi)        

In [5]:
checkHarmonic = False
log10FAP = -5.0
logFAP_limit = log10FAP
polyfit_deg = 3

spec_dir = "/Users/benjaminroulston/Dropbox/Research/TDSS/Variable_Stars/HARD_COPY_ORGINAL_DATA/SDSS_spec/02-26-2020/SDSSspec/"
CSS_LC_dir = "/Users/benjaminroulston/Dropbox/Research/TDSS/Variable_Stars/HARD_COPY_ORGINAL_DATA/CSS_LCs/csvs/"
ZTF_LC_dir = "/Users/benjaminroulston/Dropbox/Research/TDSS/Variable_Stars/HARD_COPY_ORGINAL_DATA/ZTF/DATA/07-27-2021/"

ZTF_filters = ['g', 'r']
ZTF_LC_file_names = [f'TDSS_VarStar_ZTFDR6_{ZTF_filter}_GroupID.fits' for ZTF_filter in ZTF_filters]
ZTF_g_LCs = Table.read(ZTF_LC_dir + ZTF_LC_file_names[0])
ZTF_r_LCs = Table.read(ZTF_LC_dir + ZTF_LC_file_names[1])

ZTF_LC_data = Table.read("/Users/benjaminroulston/Dropbox/Research/TDSS/Variable_Stars/HARD_COPY_ORGINAL_DATA/ZTF/DATA/07-27-2021/TDSS_VarStar_ZTFDR6_gri_GroupID.fits")

TDSS_prop = Table.read("/Users/benjaminroulston/Dropbox/Research/TDSS/Variable_Stars/HARD_COPY_ORGINAL_DATA/PROGRAM_SAMPLE/2021-07-27/FINAL_FILES/TDSS_SES+PREV_DR16DR12griLT20_GaiaEDR3_Drake2014PerVar_CSSID_ZTFIDs_LCpointer_PyHammer_EqW.fits")
TDSS_prop

ra_GaiaEDR3,dec_GaiaEDR3,plate,mjd,fiber,class,subclass,z,zErr,ELODIE_FILENAME,ELODIE_OBJECT,ELODIE_SPTYPE,ELODIE_BV,ELODIE_TEFF,ELODIE_LOGG,ELODIE_FEH,ELODIE_Z,ELODIE_Z_ERR,ELODIE_Z_MODELERR,ELODIE_RCHI2,ELODIE_DOF,Z_NOQSO,Z_ERR_NOQSO,CLASS_NOQSO,SUBCLASS_NOQSO,class_SDSSDR12,umag,e_umag,gmag,e_gmag,rmag,e_rmag,imag,e_imag,zmag,e_zmag,zsp,e_zsp,f_zsp,zph,e_zph,avg_zph,pmRA,e_pmRA,pmDE,e_pmDE,SpObjID,spType,spCl,subClass_SDSSDR12,ra_epoch2000,dec_epoch2000,ra_error,dec_error,l_GaiaEDR3,b_GaiaEDR3,Z_GaiaEDR3,U_GaiaEDR3,V_GaiaEDR3,W_GaiaEDR3,parallax,parallax_error,parallax_over_error,pm,pmra_GaiaEDR3,pmra_error,pmdec,pmdec_error,astrometric_n_good_obs_al,astrometric_gof_al,astrometric_chi2_al,astrometric_excess_noise,astrometric_excess_noise_sig,astrometric_params_solved,pseudocolour,pseudocolour_error,visibility_periods_used,ruwe,duplicated_source,phot_g_mean_flux,phot_g_mean_flux_error,phot_g_mean_mag,phot_bp_mean_flux,phot_bp_mean_flux_error,phot_bp_mean_mag,phot_rp_mean_flux,phot_rp_mean_mag,phot_bp_rp_excess_factor,bp_rp,dr2_radial_velocity,dr2_radial_velocity_error,dr2_rv_nb_transits,dr2_rv_template_teff,dr2_rv_template_logg,panstarrs1,sdssdr13,skymapper2,urat1,phot_g_mean_mag_error,phot_bp_mean_mag_error,phot_rp_mean_mag_error,phot_g_mean_mag_corrected,phot_g_mean_mag_error_corrected,phot_g_mean_flux_corrected,phot_bp_rp_excess_factor_corrected,ra_epoch2000_error,dec_epoch2000_error,ra_dec_epoch2000_corr,rgeo,b_rgeo_GaiaEDR3,B_rgeo_GaiaEDR3a,rpgeo,b_rpgeo_GaiaEDR3,B_rpgeo_GaiaEDR3a,isDrake,Drake_CRTS,Drake_Vmag,Drake_Per,Drake_Vamp,Drake_Cl,Drake_f_Cl,Drake_Simbad,CSSID,CSS_Nepochs,CSS_mag,CSS_magerr,ZTF_GroupID,ZTF_g_Nepochs,ZTF_g_mag,ZTF_g_magerr,ZTF_r_Nepochs,ZTF_r_mag,ZTF_r_magerr,ZTF_i_Nepochs,ZTF_i_mag,ZTF_i_magerr,CSSLC,ZTFLC,CSS_or_ZTF,CSS_and_ZTF,PyHammerSpecType,PyHammerRV,CaK,CaK_var,Cadel,Cadel_var,CaI4217,CaI4217_var,Gband,Gband_var,Hgam,Hgam_var,FeI4383,FeI4383_var,FeI4404,FeI4404_var,Hbeta,Hbeta_var,MgI,MgI_var,NaD,NaD_var,CaI6162,CaI6162_var,Halpha,Halpha_var,CaH2,CaH2_var,CaH3,CaH3_var,TiO5,TiO5_var,VO7434,VO7434_var,VO7445,VO7445_var,VO-B,VO-B_var,VO7912,VO7912_var,Rb-B,Rb-B_var,NaI,NaI_var,TiO8,TiO8_var,TiO8440,TiO8440_var,Cs-A,Cs-A_var,CaII8498,CaII8498_var,CrH-A,CrH-A_var,CaII8662,CaII8662_var,FeI8689,FeI8689_var,FeH,FeH_var,C2-4382,C2-4382_var,C2-4737,C2-4737_var,C2-5165,C2-5165_var,C2-5636,C2-5636_var,CN-6959,CN-6959_var,CN-7088,CN-7088_var,CN-7259,CN-7259_var,CN-7820,CN-7820_var,CN-8067,CN-8067_var,CN-8270,CN-8270_var,WD-Halpha,WD-Halpha_var,WD-Hbeta,WD-Hbeta_var,WD-Hgamma,WD-Hgamma_var,W_Halpha,W_Halpha_error
deg,deg,,,,,,,,,,,,,,,,,,,,,,,,,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,,,,,,,mas / yr,mas / yr,mas / yr,mas / yr,,,,,deg,deg,mas,mas,deg,deg,pc,km / s,km / s,km / s,mas,mas,,mas / yr,mas / yr,mas / yr,mas / yr,mas / yr,,,,mas,,,1 / um,1 / um,,,,e-/s,e-/s,mag,e-/s,e-/s,mag,e-/s,mag,,mag,km / s,km / s,,K,[cgs],,,,,mag,mag,mag,mag,mag,e-/s,,mas,mas,,pc,pc,pc,pc,pc,pc,,,mag,d,mag,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,Angstrom,Angstrom
float64,float64,int32,int32,int32,bytes6,bytes21,float32,float32,bytes16,bytes9,bytes12,float32,float32,float32,float32,float32,float32,float32,float32,int32,float32,float32,bytes6,bytes21,uint8,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int16,float64,float64,float64,float64,float64,float64,float64,int64,bytes19,bytes6,bytes21,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int16,float64,float64,float64,float64,uint8,float64,float64,uint8,float64,bool,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int16,float32,float32,int64,int64,int32,int32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,bool,bytes16,float32,float64,float

In [6]:
selected_props = ['Tspan100', 'Tspan95', 'a95', 'Mt', 'lc_skew', 'Chi2', 'brtcutoff', 'brt10per', 'fnt10per', 'fntcutoff', 'errmn', 'ferrmn', 'ngood', 'nrejects', 'nabove', 'nbelow', 'VarStat', 'Con', 'm', 'b_lin', 'chi2_lin', 'a', 'b_quad', 'c', 'chi2_quad', 'P', 'omega_best', 'is_Periodic', 'logProb', 'Amp', 'isAlias', 'time_whittened', 't0', 'power']

properties = TDSS_prop.copy()
colnames = properties.colnames
colnames.remove('ra_GaiaEDR3')
colnames.remove('dec_GaiaEDR3')
properties.remove_columns(colnames)
properties

ra_GaiaEDR3,dec_GaiaEDR3
deg,deg
float64,float64
0.01307073795,-2.43008296623
0.01470748805,20.15233374844
0.02193203895,1.5599789031
0.02716434631,26.08194970618
0.03463319489,0.52495551063
0.03717687973,25.89781739728
0.04025346014,5.77216037972
0.04434511248,20.12578967848


In [ ]:
def TDSS_LC_ANALYSIS(prop_id):
    ROW = TDSS_prop[prop_id]
    is_CSS = ROW['CSSLC']
    is_ZTF = ROW['ZTFLC']
    
    object_ra = ROW['ra_GaiaEDR3']
    object_dec = ROW['dec_GaiaEDR3']
    ra_string = '{:0>9.5f}'.format(object_ra)
    dec_string = '{:0=+9.5f}'.format(object_dec)
    
    c = ICRS(object_ra*u.degree, object_dec*u.degree)
    rahmsstr = c.ra.to_string(u.hour, precision=2, pad=True)
    decdmsstr = c.dec.to_string(u.degree, alwayssign=True, precision=2, pad=True)

    this_filename_base = f"{ra_string}{dec_string}_"
    
    if is_CSS:
        if ROW[f"CSS_Nepochs"] > 10:        
            lc_file = CSS_LC_dir+str(ROW['CSSID'])+'.dat'
            CSS_lc_data = Table.read(lc_file, format='ascii', names=['mjd', 'mag', 'magerr'])
            popt = np.array([-2.61242938e+01,  1.93636204e+00,  4.45971381e-01, -6.49419310e-02, 2.99231126e-03,  2.40758201e-01, -2.40805035e-01])
            magerr_resid_mean = 0.008825118765717422
            shift_const = 1.5 * magerr_resid_mean
            pred_magerr = low_order_poly(CSS_lc_data['mag'], popt[0], popt[1], popt[2], popt[3], popt[4], popt[5], popt[6])
            bad_err_index = np.where(CSS_lc_data['magerr'] - (pred_magerr - shift_const) < 0.0)[0]
            CSS_lc_data['magerr'][bad_err_index] = pred_magerr[bad_err_index]
            
            mean_mag = np.nanmean(CSS_lc_data['mag'])
            
            flc_data, LC_stat_properties = LCtools.process_LC(CSS_lc_data, fltRange=5.0, detrend=True)

            LC_period_properties, all_period_properties = LCtools.perdiodSearch(flc_data, minP=0.1, maxP=100.0, log10FAP=log10FAP, checkHarmonic=False)
            all_period_properties = {**LC_stat_properties, **all_period_properties}
            #LC_prop = {**LC_stat_properties, **LC_period_properties}
            
            FAP_power_peak = all_period_properties['ls'].false_alarm_level(10**log10FAP)
            #new_props = { ii: all_period_properties[ii] for ii in selected_props }
            new_props = {**all_period_properties, **{'FAP_power_peak':FAP_power_peak}}
      
            #pickle_filename = raw_LC_analysis_dir_CSS + f"{prop_id}_CSS_{ROW['CSSID']}.xz"
            #with open(pickle_filename, 'wb') as f:
            #    pickle.dump(new_props, f)   
            #with lzma.open(pickle_filename, "wb") as f:
            #    pickle.dump(new_props, f)
                
            pickled_data = pickle.dumps(new_props)  # returns data as a bytes object
            compressed_pickle = blosc.compress(pickled_data)
            pickle_filename = raw_LC_analysis_dir_CSS + f"{prop_id}_CSS_{ROW['CSSID']}.dat"
            with open(pickle_filename, "wb") as f:
                f.write(compressed_pickle)

    if is_ZTF:
        for ii, this_ZTF_filter in enumerate(['g', 'r', 'i']):
            if ROW[f"ZTF_{this_ZTF_filter}_Nepochs"] > 10:
                lc_index = (ZTF_LC_data['ZTF_GroupID'] == ROW['ZTF_GroupID']) & (ZTF_LC_data['filtercode'] == 'z'+this_ZTF_filter)
                lc_data = ZTF_LC_data[lc_index]
                
                mean_mag = np.nanmean(lc_data['mag'])
            
                flc_data, LC_stat_properties = LCtools.process_LC(lc_data, fltRange=5.0, detrend=True)
                
                LC_period_properties, all_period_properties = LCtools.perdiodSearch(flc_data, minP=0.1, maxP=100.0, log10FAP=log10FAP, checkHarmonic=False)
                all_period_properties = {**LC_stat_properties, **all_period_properties}
                #LC_prop = {**LC_stat_properties, **LC_period_properties}
                
                FAP_power_peak = all_period_properties['ls'].false_alarm_level(10**log10FAP)
                #new_props = { ii: all_period_properties[ii] for ii in selected_props }
                new_props = {**all_period_properties, **{'FAP_power_peak':FAP_power_peak}}

                #pickle_filename = raw_lc_analysis_dir_ZTF + f"/{this_ZTF_filter}/" + f"{prop_id}_ZTF{this_ZTF_filter}_{ROW['ZTF_GroupID']}.xz"
                #with open(pickle_filename, 'wb') as f:
                #    pickle.dump(new_props, f) 
                #with lzma.open(pickle_filename, "wb") as f:
                #    pickle.dump(new_props, f)
                    
                pickled_data = pickle.dumps(new_props)  # returns data as a bytes object
                compressed_pickle = blosc.compress(pickled_data)
                pickle_filename = raw_lc_analysis_dir_ZTF + f"/{this_ZTF_filter}/" + f"{prop_id}_ZTF{this_ZTF_filter}_{ROW['ZTF_GroupID']}.xz"
                with open(pickle_filename, "wb") as f:
                    f.write(compressed_pickle)
                
# for prop_id in tqdm.tqdm(range(len(TDSS_prop))):
#     TDSS_LC_ANALYSIS(prop_id)
    
if __name__ == '__main__':
    with Pool(os.cpu_count()-2) as pool:
        r = list(tqdm.tqdm(pool.imap(TDSS_LC_ANALYSIS, range(len(TDSS_prop))), total=len(TDSS_prop)))
        
# pool = Pool(os.cpu_count()-2) # Create a multiprocessing Pool
# pool.map(TDSS_LC_ANALYSIS, range(len(TDSS_prop)))

In [9]:
blank_4list = [{}, {}, {}, {}]
all_prop = [blank_4list.copy() for ii in TDSS_prop]

for prop_id, ROW in enumerate(tqdm.tqdm(TDSS_prop)):
    is_CSS = ROW['CSSLC']
    is_ZTF = ROW['ZTFLC']
    
    object_ra = ROW['ra_GaiaEDR3']
    object_dec = ROW['dec_GaiaEDR3']
    ra_string = '{:0>9.5f}'.format(object_ra)
    dec_string = '{:0=+9.5f}'.format(object_dec)
    
    c = ICRS(object_ra*u.degree, object_dec*u.degree)
    rahmsstr = c.ra.to_string(u.hour, precision=2, pad=True)
    decdmsstr = c.dec.to_string(u.degree, alwayssign=True, precision=2, pad=True)

    this_filename_base = f"{ra_string}{dec_string}_"
    
    if is_CSS:
        if ROW[f"CSS_Nepochs"] > 10:        
            lc_file = CSS_LC_dir+str(ROW['CSSID'])+'.dat'
            CSS_lc_data = Table.read(lc_file, format='ascii', names=['mjd', 'mag', 'magerr'])
            popt = np.array([-2.61242938e+01,  1.93636204e+00,  4.45971381e-01, -6.49419310e-02, 2.99231126e-03,  2.40758201e-01, -2.40805035e-01])
            magerr_resid_mean = 0.008825118765717422
            shift_const = 1.5 * magerr_resid_mean
            pred_magerr = low_order_poly(CSS_lc_data['mag'], popt[0], popt[1], popt[2], popt[3], popt[4], popt[5], popt[6])
            bad_err_index = np.where(CSS_lc_data['magerr'] - (pred_magerr - shift_const) < 0.0)[0]
            CSS_lc_data['magerr'][bad_err_index] = pred_magerr[bad_err_index]
            
            mean_mag = np.nanmean(CSS_lc_data['mag'])
            
            flc_data, LC_stat_properties = LCtools.process_LC(CSS_lc_data, fltRange=5.0, detrend=True)

            LC_period_properties, all_period_properties = LCtools.perdiodSearch(flc_data, minP=0.1, maxP=100.0, log10FAP=log10FAP, checkHarmonic=False)
            all_period_properties = {**LC_stat_properties, **all_period_properties}
            #LC_prop = {**LC_stat_properties, **LC_period_properties}
            
            FAP_power_peak = all_period_properties['ls'].false_alarm_level(10**log10FAP)
            new_props = { ii: all_period_properties[ii] for ii in selected_props }
            new_props = {**new_props, **{'FAP_power_peak':FAP_power_peak}}

            all_prop[prop_id][0] = new_props#all_period_properties
            #plot_VARSTAR_LC(flc_data, all_period_properties, log10FAP)
            #plt.savefig(lc_dir_CSS + "/" + this_filename_base + f"CSS_{ROW['CSSID']}_LC.pdf", dpi=600)
            #plt.clf()
            #plt.close()

    if is_ZTF:
        for ii, this_ZTF_filter in enumerate(['g', 'r', 'i']):
            if ROW[f"ZTF_{this_ZTF_filter}_Nepochs"] > 10:
                lc_index = (ZTF_LC_data['ZTF_GroupID'] == ROW['ZTF_GroupID']) & (ZTF_LC_data['filtercode'] == 'z'+this_ZTF_filter)
                lc_data = ZTF_LC_data[lc_index]
                
                mean_mag = np.nanmean(lc_data['mag'])
            
                flc_data, LC_stat_properties = LCtools.process_LC(lc_data, fltRange=5.0, detrend=True)
                
                LC_period_properties, all_period_properties = LCtools.perdiodSearch(flc_data, minP=0.1, maxP=100.0, log10FAP=log10FAP, checkHarmonic=False)
                all_period_properties = {**LC_stat_properties, **all_period_properties}
                #LC_prop = {**LC_stat_properties, **LC_period_properties}
                
                FAP_power_peak = all_period_properties['ls'].false_alarm_level(10**log10FAP)
                new_props = { ii: all_period_properties[ii] for ii in selected_props }
                new_props = {**new_props, **{'FAP_power_peak':FAP_power_peak}}

                all_prop[prop_id][ii+1] = new_props #all_period_properties
                #save_dir = [lc_dir_ZTFg, lc_dir_ZTFr, lc_dir_ZTFi][ii]
                #plot_VARSTAR_LC(flc_data, all_period_properties, log10FAP)
                #plt.savefig(save_dir + "/" + this_filename_base + f"ZTF_{this_ZTF_filter}_LC.pdf", dpi=600)
                #plt.clf()
                #plt.close()

                
pickle_filename = 'TDSS_VarStar_allprop_09-07-2021.pkl'
with open(pickle_filename, 'wb') as f:
    pickle.dump(all_prop, f)

with open(pickle_filename, 'rb') as f:
    all_prop = pickle.load(f)

  0%|          | 0/25121 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [7]:
################################################################################
################################################################################
################################################################################
def plot_VARSTAR_LC(flc_data, all_period_properties, log10FAP):
    fig = plt.figure(figsize=(13, 9), constrained_layout=True)
    outer_gs = GridSpec(2, 2, figure=fig, height_ratios=[1, 1], width_ratios=[1, 1], hspace=0.0)

    ax1 = fig.add_subplot(outer_gs[0, 0])

    inner_grid1 = GridSpecFromSubplotSpec(2, 1, subplot_spec=outer_gs[0, 1], hspace=0.0, width_ratios=[1], height_ratios=[0.75, 0.25])
    inner_grid2 = GridSpecFromSubplotSpec(2, 1, subplot_spec=outer_gs[1, :], hspace=0.0, width_ratios=[1], height_ratios=[1, 1])

    ax2 = fig.add_subplot(inner_grid1[0])
    ax25 = fig.add_subplot(inner_grid1[1], sharex=ax2)
    ax3 = fig.add_subplot(inner_grid2[0])
    ax4 = fig.add_subplot(inner_grid2[1], sharex=ax3)

    plt.setp(ax2.get_xticklabels(), visible=False)
    plt.setp(ax3.get_xticklabels(), visible=False)
    ################################################################################
    ################################################################################
    FAP_power_peak = all_period_properties['ls'].false_alarm_level(10**log10FAP)
    plot_single_windowfun(flc_data.copy(), ax3)
    plot_single_powerspec(all_period_properties['frequency'], all_period_properties['power'], P=all_period_properties['P'], ax1=ax4, FAP_power_peak=FAP_power_peak, logFAP_limit=log10FAP, title="")

    plt_any_lc_fig(flc_data, all_period_properties['P'], ax2, ax25, is_Periodic=True, title="", phasebin=False, bins=25, phasebinonly=False, show_err_lines=True, plot_rejected=False)
    plt_raw_lc(flc_data, ax1, title="", show_err_lines=True, plot_rejected=True)

    ################################################################################
    fig.suptitle(f'RA= {ra_string} DEC= {dec_string} $|$ ZTF {this_ZTF_filter}', fontsize=16)
    
    axes = [ax1, ax2, ax25, ax3, ax4]
    return fig, axes

# plt.savefig(this_filename_base+"ZTFLC.pdf", dpi=600)

In [8]:
def low_order_poly(mag, a, b, c, d, e, f_, g):
    return a + b * mag + c * mag**2 + d * mag**3 + e * mag**4 + f_ * mag**5 + g * mag**5


def plt_raw_lc(lc_data, ax, title="", show_err_lines=True, plot_rejected=True):
    goodQualIndex = np.where(lc_data['QualFlag']==True)[0]
    badQualIndex = np.where(lc_data['QualFlag']==False)[0]
    mjd = lc_data['mjd'][goodQualIndex].data
    mag = lc_data['mag'][goodQualIndex].data
    err = lc_data['magerr'][goodQualIndex].data

    mjd_bad = lc_data['mjd'][badQualIndex].data
    mag_bad = lc_data['mag'][badQualIndex].data
    err_bad = lc_data['magerr'][badQualIndex].data

    if title != "":
        ax.set_title(title)

    fmagmn = np.mean(mag)
    ferrmn = np.mean(err)
    fmag_stdev = np.std(mag)

    if show_err_lines:
        ax.axhline(fmagmn, color='r', ls='-', lw=1.5, alpha=0.5)
        
        ax.axhline(fmagmn + 3 * ferrmn, color='g', ls='-.', lw=1.5, alpha=0.5, label='3X Mag Err')
        ax.axhline(fmagmn - 3 * ferrmn, color='g', ls='-.', lw=1.5, alpha=0.5)
        ax.axhline(fmagmn + 3 * fmag_stdev, color='b', ls=':', lw=1.5, alpha=0.5, label='3X Mag StDev')
        ax.axhline(fmagmn - 3 * fmag_stdev, color='b', ls=':', lw=1.5, alpha=0.5)


        ax.errorbar(mjd, mag, err, fmt='.k', ecolor='gray', lw=1, ms=4, capsize=1.5, alpha=1.0)
        if plot_rejected:
            ax.errorbar(mjd_bad, mag_bad, err_bad, fmt='.r', ecolor='r', lw=1, ms=4, capsize=1.5, alpha=1.0)

        ax.set_xlabel('MJD')
        ax.grid()
        ax.set_ylabel('mag')
        ax.invert_yaxis()
    return ax

def plot_single_windowfun(flc_data, ax, title='Window Function', P=None):
    goodQualIndex = np.where(flc_data['QualFlag'] == 1)[0]
    lc_mjd = flc_data['mjd'][goodQualIndex]
    lc_mag = flc_data['mag'][goodQualIndex]
    lc_err = flc_data['magerr'][goodQualIndex]

    t_days = lc_mjd  # * u.day
    y_mags = lc_mag  # * u.mag
    dy_mags = lc_err  # * u.mag

    if t_days.unit == None:
        t_days = t_days * u.d
    if y_mags.unit == None:
        y_mags = y_mags * u.mag
    if dy_mags.unit == None:
        dy_mags = dy_mags * u.mag

    y_mags = y_mags / y_mags.value

    minP =  0.1 * u.d
    maximum_frequency = (minP)**-1
    frequency = np.linspace(0, maximum_frequency.value, num=250001)[1:] / u.d

    ls = LombScargle(t_days, y_mags, dy_mags, fit_mean=False, center_data=False)
    power = ls.power(frequency=frequency)

    title = ""
    plot_single_powerspec(frequency, power, ax1=ax, title=title, window=True, P=P)
    #ymin, ymax = ax1.get_ylim()
    #ymin - (0.11*ymax)
    # ax1.annotate('',
    #          xy=(((use_P*u.d)**-1).to(1/u.d).value, ymin),
    #          xytext=(((use_P*u.d)**-1).to(1/u.d).value, ymin - (0.11*ymax)),
    #          xycoords='data', annotation_clip=False, arrowprops=dict(arrowstyle="->", color='r'))
    #ax.tick_params(axis='both', which='major', labelsize=15, width=2.0, length=10)
    #ax.tick_params(axis='both', which='minor', width=1.0, length=5)

    #ax.xaxis.label.set_size(15)
    #ax.yaxis.label.set_size(15)
    #ax.set_xlabel('')
    return ax


def plot_single_powerspec(frequency, power, P, ax1=None, FAP_power_peak=None, logFAP_limit=None, alias_df=None, title="", window=False):
    minimum_frequency = frequency.min()
    maximum_frequency = frequency.max()

    minP = maximum_frequency**-1
    maxP = minimum_frequency**-1

    if (ax1 is None):
        fig = plt.figure(figsize=(12, 4), constrained_layout=True, dpi=600)
        gs = GridSpec(1, 1, figure=fig)
        ax1 = fig.add_subplot(gs[0, 0])

    ax1.set_title(title)
    ax1.plot(frequency.to(1/u.d), power, c='k', lw=0.75)
    if FAP_power_peak is not None:
        ax1.axhline(y=FAP_power_peak, c='r', ls='dashed', alpha=0.5, lw=0.75)
        if logFAP_limit is not None:
            ax1.text(0.8 * maximum_frequency.to(1/u.d).value, FAP_power_peak + 0.0, f"log(FAP) = {logFAP_limit}", c='r')

    xmin = minimum_frequency.to(1/u.d).value
    xmax = maximum_frequency.to(1/u.d).value

    #ax1.axvline(x=((365.25 * u.d)**-1).to(1/u.d).value, c='k', ls='dashed', alpha=0.5, lw=0.75)
    #ax1.axvline(x=((29.530587981 * u.d)**-1).to(1/u.d).value, c='k', ls='dashed', alpha=0.5, lw=0.75)
    #ax1.axvline(x=((27.321661 * u.d)**-1).to(1/u.d).value, c='k', ls='dashed', alpha=0.5, lw=0.75)
    #ax1.axvline(x=((9 * u.d)**-1).to(1/u.d).value, c='k', ls='dashed', alpha=0.5, lw=0.75)
    #ax1.axvline(x=((8 * u.d)**-1).to(1/u.d).value, c='k', ls='dashed', alpha=0.5, lw=0.75)
    #ax1.axvline(x=((7 * u.d)**-1).to(1/u.d).value, c='k', ls='dashed', alpha=0.5, lw=0.75)
    #ax1.axvline(x=((6 * u.d)**-1).to(1/u.d).value, c='k', ls='dashed', alpha=0.5, lw=0.75)
    #ax1.axvline(x=((5 * u.d)**-1).to(1/u.d).value, c='k', ls='dashed', alpha=0.5, lw=0.75)
    #ax1.axvline(x=((4 * u.d)**-1).to(1/u.d).value, c='k', ls='dashed', alpha=0.5, lw=0.75)
    #ax1.axvline(x=((3 * u.d)**-1).to(1/u.d).value, c='k', ls='dashed', alpha=0.5, lw=0.75)
    #ax1.axvline(x=((2 * u.d)**-1).to(1/u.d).value, c='k', ls='dashed', alpha=0.5, lw=0.75)
    
    ax1.axvline(x=((1 * u.d)**-1).to(1/u.d).value, c='k', ls='dashed', alpha=0.5, lw=0.75)
    ax1.axvline(x=((1 / 2 * u.d)**-1).to(1/u.d).value, c='k', ls='dashed', alpha=0.5, lw=0.75)
    ax1.axvline(x=((1 / 3 * u.d)**-1).to(1/u.d).value, c='k', ls='dashed', alpha=0.5, lw=0.75)
    ax1.axvline(x=((1 / 4 * u.d)**-1).to(1/u.d).value, c='k', ls='dashed', alpha=0.5, lw=0.75)
    ax1.axvline(x=((1 / 5 * u.d)**-1).to(1/u.d).value, c='k', ls='dashed', alpha=0.5, lw=0.75)
    ax1.axvline(x=((1 / 6 * u.d)**-1).to(1/u.d).value, c='k', ls='dashed', alpha=0.5, lw=0.75)
    ax1.axvline(x=((1 / 7 * u.d)**-1).to(1/u.d).value, c='k', ls='dashed', alpha=0.5, lw=0.75)
    ax1.axvline(x=((1 / 8 * u.d)**-1).to(1/u.d).value, c='k', ls='dashed', alpha=0.5, lw=0.75)
    ax1.axvline(x=((1 / 9 * u.d)**-1).to(1/u.d).value, c='k', ls='dashed', alpha=0.5, lw=0.75)
    ax1.set_xlabel(r'Frequency [d$^{-1}$]')
    ax1.set_ylabel('Power')
    ax1.set_xlim((xmin, xmax))

    xmin = minimum_frequency.to(1/u.d).value
    xmax = maximum_frequency.to(1/u.d).value
    dx = xmax - xmin
    np.ceil(dx)

    xmajortick = 1 #np.floor(np.ceil(dx) / 23)
    xminortick = xmajortick / 10

    ax1.xaxis.set_major_locator(ticker.MultipleLocator(xmajortick))
    ax1.xaxis.set_minor_locator(ticker.MultipleLocator(xminortick))

    ax1.yaxis.set_major_locator(ticker.MultipleLocator(0.2))
    ax1.yaxis.set_minor_locator(ticker.MultipleLocator(0.02))
    
    #ax1.tick_params(axis='both', which='major', labelsize=15, width=2.0, length=10)
    #ax1.tick_params(axis='both', which='minor', width=1.0, length=5)

    #ax1.xaxis.label.set_size(15)
    #ax1.yaxis.label.set_size(15)
    
    #ax1.set_xscale('log')
    if window==False:
        ymin, ymax = ax1.get_ylim()
        ax1.annotate('',
             xy=(((P*u.d)**-1).to(1/u.d).value, ymin),
             xytext=(((P*u.d)**-1).to(1/u.d).value, ymin - (0.11*ymax)),
             xycoords='data', annotation_clip=False, arrowprops=dict(arrowstyle="->", color='r'))

    if alias_df is not None:
        n = [-4, -3, -2, -1, 1, 2, 3, 4]
        f0 = frequency[np.argmax(power)]
        ax1.axvline(x=f0.to(1/u.d).value, c='r', ls='dashed', alpha=0.75, lw=0.75)
        for ii in n:
            ax1.axvline(x=np.abs((f0 + (ii * alias_df)).to(1/u.d).value), c='b', ls='dashed', alpha=0.5, lw=0.75)
            if ii > 1:
                ax1.axvline(x=(f0 / ii).to(1/u.d).value, c='g', ls='dashed', alpha=0.5, lw=0.75)
                ax1.axvline(x=(f0 * ii).to(1/u.d).value, c='g', ls='dashed', alpha=0.5, lw=0.75)

    return ax1

def plt_any_lc(lc_data, P, is_Periodic=False, figsize=(8, 3), title="", phasebin=False, bins=25, phasebinonly=False, show_err_lines=True, plot_rejected=False):
    fig = plt.figure(figsize=figsize, constrained_layout=False, dpi=600)
    gs = GridSpec(4, 1, figure=fig)
    ax1 = fig.add_subplot(gs[0:3, :])
    ax2 = fig.add_subplot(gs[3, :], sharex=ax1)

    gs.update(hspace=0.0) # set the spacing between axes. 
    plt.setp(ax1.get_xticklabels(), visible=False)



    goodQualIndex = np.where(lc_data['QualFlag']==True)[0]
    badQualIndex = np.where(lc_data['QualFlag']==False)[0]
    mjd = lc_data['mjd'][goodQualIndex].data
    mag = lc_data['mag'][goodQualIndex].data
    err = lc_data['magerr'][goodQualIndex].data

    data = [mjd, mag, err]
    AFD_data = LCtools.AFD(data, P)

    Nterms, phase_fit, y_fit, phased_t, resid, reduced_ChiS, mfit = AFD_data

    Amp = y_fit.max() - y_fit.min()
    t0 = (mjd - (phased_t * P)).min()

    title = title  # + "Amp = {!s} $|$ t0 = {!s}".format(np.round(Amp, 3), np.round(t0, 7))

    mjd_bad = lc_data['mjd'][badQualIndex].data
    phase_bad = ((mjd_bad - t0) / P) % 1
    mag_bad = lc_data['mag'][badQualIndex].data
    err_bad = lc_data['magerr'][badQualIndex].data

    binned_phase, binned_mag, binned_err = LCtools.bin_phaseLC(phased_t, mag, err, bins=bins)

    if title is not "":
        ax1.set_title(title)
    # is_Periodic = True
    if is_Periodic:
        if phasebinonly:
            pass
        else:
            ax1.errorbar(phased_t, mag, err, fmt='.k', ecolor='k', lw=1, ms=4, capsize=0, alpha=0.750, elinewidth=0.25)
            ax1.errorbar(phased_t + 1, mag, err, fmt='.k', ecolor='k', lw=1, ms=4, capsize=0, alpha=0.75, elinewidth=0.25)

            if plot_rejected:
                ax1.errorbar(phase_bad, mag_bad, err_bad, fmt='.r', ecolor='r', lw=1, ms=4, capsize=0, alpha=0.5)
                ax1.errorbar(phase_bad + 1, mag_bad, err_bad, fmt='.r', ecolor='r', lw=1, ms=4, capsize=0, alpha=0.5)

        ax1.plot(phase_fit, y_fit, 'r', markeredgecolor='r', lw=0.5, fillstyle='top', linestyle='solid', zorder=10)
        ax1.plot(phase_fit + 1, y_fit, 'r', markeredgecolor='r', lw=0.5, fillstyle='top', linestyle='solid', zorder=10)
        # plt_ax.plot(phase_fit+2, y_fit, 'b', markeredgecolor='b', lw=2, fillstyle='top', linestyle='solid')

        fmagmn = np.mean(mag)
        ferrmn = np.mean(err)
        fmag_stdev = np.std(mag)

        #ax1.axhline(fmagmn, color='r', ls='-', lw=0.5, label='Mean Mag')
        if show_err_lines:
            ax1.axhline(fmagmn + 3 * ferrmn, color='g', ls='-.', lw=0.5, alpha=0.5, label='3X Mag Err')
            ax1.axhline(fmagmn - 3 * ferrmn, color='g', ls='-.', lw=0.5, alpha=0.5)
            ax1.axhline(fmagmn + 3 * fmag_stdev, color='b', ls=':', lw=0.5, alpha=0.5, label='3X Mag StDev')
            ax1.axhline(fmagmn - 3 * fmag_stdev, color='b', ls=':', lw=0.5, alpha=0.5)
        else:
            pass

        ax1.set_ylabel('mag')
        # plt_ax.set_xlabel('Phase')
        ax1.invert_yaxis()

        ax1.grid()
        if phasebin:
            ax1.errorbar(binned_phase, binned_mag, binned_err, fmt='sr', ecolor='red', lw=1, ms=4, capsize=0, alpha=0.3, zorder=10)
            ax1.errorbar(binned_phase + 1, binned_mag, binned_err, fmt='sr', ecolor='red', lw=1, ms=4, capsize=0, alpha=0.3, zorder=10)

        ax2.set_xlim(0.0, 2.0)

        ax2.errorbar(phased_t, resid, fmt='.k', ecolor='k', lw=1, ms=4, capsize=1.5, alpha=1.0)
        ax2.errorbar(phased_t + 1, resid, fmt='.k', ecolor='k', lw=1, ms=4, capsize=1.5, alpha=1.0)

        ax2.grid()
        ax2.set_xlim(0.0, 2.0)

        ax2.set_xlabel(r'Phase')
        ax2.set_ylabel('Residual')  # \n N$_{terms} = 4$')
    else:
        ax1.errorbar(mjd, mag, err, fmt='.k', ecolor='gray', lw=1, ms=4, capsize=1.5, alpha=1.0)
        if plot_rejected:
            ax1.errorbar(mjd_bad, mag_bad, err_bad, fmt='.r', ecolor='r', lw=1, ms=4, capsize=1.5, alpha=1.0)

        fmagmn = np.mean(mag)
        ferrmn = np.mean(err)
        fmag_stdev = np.std(mag)

        ax1.axhline(fmagmn, color='r', ls='-', lw=2, label='Mean Mag')
        ax1.axhline(fmagmn + 3 * ferrmn, color='g', ls='-.', lw=2, alpha=0.5, label='3X Mag Err')
        ax1.axhline(fmagmn - 3 * ferrmn, color='g', ls='-.', lw=2, alpha=0.5)
        ax1.axhline(fmagmn + 3 * fmag_stdev, color='b', ls=':', lw=2, alpha=0.5, label='3X Mag StDev')
        ax1.axhline(fmagmn - 3 * fmag_stdev, color='b', ls=':', lw=2, alpha=0.5)

        ax1.set_xlabel('MJD')
        ax1.grid()
        ax1.set_ylabel('mag')
        ax1.invert_yaxis()
    return ax1, ax2



def plt_any_lc_fig(lc_data, P, ax1, ax2, is_Periodic=False, title="", phasebin=False, bins=25, phasebinonly=False, show_err_lines=True, plot_rejected=False):
    #fig = plt.figure(figsize=figsize, constrained_layout=False, dpi=600)
    #gs = GridSpec(4, 1, figure=fig)
    #ax1 = fig.add_subplot(gs[0:3, :])
    #ax2 = fig.add_subplot(gs[3, :], sharex=ax1)

    #gs.update(hspace=0.0) # set the spacing between axes. 
    #plt.setp(ax1.get_xticklabels(), visible=False)



    goodQualIndex = np.where(lc_data['QualFlag']==True)[0]
    badQualIndex = np.where(lc_data['QualFlag']==False)[0]
    mjd = lc_data['mjd'][goodQualIndex].data
    mag = lc_data['mag'][goodQualIndex].data
    err = lc_data['magerr'][goodQualIndex].data

    data = [mjd, mag, err]
    AFD_data = LCtools.AFD(data, P)

    Nterms, phase_fit, y_fit, phased_t, resid, reduced_ChiS, mfit = AFD_data

    Amp = y_fit.max() - y_fit.min()
    t0 = (mjd - (phased_t * P)).min()

    title = title  # + "Amp = {!s} $|$ t0 = {!s}".format(np.round(Amp, 3), np.round(t0, 7))

    mjd_bad = lc_data['mjd'][badQualIndex].data
    phase_bad = ((mjd_bad - t0) / P) % 1
    mag_bad = lc_data['mag'][badQualIndex].data
    err_bad = lc_data['magerr'][badQualIndex].data

    binned_phase, binned_mag, binned_err = LCtools.bin_phaseLC(phased_t, mag, err, bins=bins)

    if title is not "":
        ax1.set_title(title)
    # is_Periodic = True
    if is_Periodic:
        if phasebinonly:
            pass
        else:
            ax1.errorbar(phased_t, mag, err, fmt='.k', ecolor='k', lw=1, ms=4, capsize=0, alpha=0.750, elinewidth=0.25)
            ax1.errorbar(phased_t + 1, mag, err, fmt='.k', ecolor='k', lw=1, ms=4, capsize=0, alpha=0.75, elinewidth=0.25)

            if plot_rejected:
                ax1.errorbar(phase_bad, mag_bad, err_bad, fmt='.r', ecolor='r', lw=1, ms=4, capsize=0, alpha=0.5)
                ax1.errorbar(phase_bad + 1, mag_bad, err_bad, fmt='.r', ecolor='r', lw=1, ms=4, capsize=0, alpha=0.5)

        ax1.plot(phase_fit, y_fit, 'r', markeredgecolor='r', lw=0.5, fillstyle='top', linestyle='solid', zorder=10)
        ax1.plot(phase_fit + 1, y_fit, 'r', markeredgecolor='r', lw=0.5, fillstyle='top', linestyle='solid', zorder=10)
        # plt_ax.plot(phase_fit+2, y_fit, 'b', markeredgecolor='b', lw=2, fillstyle='top', linestyle='solid')

        fmagmn = np.mean(mag)
        ferrmn = np.mean(err)
        fmag_stdev = np.std(mag)

        #ax1.axhline(fmagmn, color='r', ls='-', lw=0.5, label='Mean Mag')
        if show_err_lines:
            ax1.axhline(fmagmn + 3 * ferrmn, color='g', ls='-.', lw=0.5, alpha=0.5, label='3X Mag Err')
            ax1.axhline(fmagmn - 3 * ferrmn, color='g', ls='-.', lw=0.5, alpha=0.5)
            ax1.axhline(fmagmn + 3 * fmag_stdev, color='b', ls=':', lw=0.5, alpha=0.5, label='3X Mag StDev')
            ax1.axhline(fmagmn - 3 * fmag_stdev, color='b', ls=':', lw=0.5, alpha=0.5)
        else:
            pass

        ax1.set_ylabel('mag')
        # plt_ax.set_xlabel('Phase')
        ax1.invert_yaxis()

        ax1.grid()
        if phasebin:
            ax1.errorbar(binned_phase, binned_mag, binned_err, fmt='sr', ecolor='red', lw=1, ms=4, capsize=0, alpha=0.3, zorder=10)
            ax1.errorbar(binned_phase + 1, binned_mag, binned_err, fmt='sr', ecolor='red', lw=1, ms=4, capsize=0, alpha=0.3, zorder=10)

        ax2.set_xlim(0.0, 2.0)

        ax2.errorbar(phased_t, resid, fmt='.k', ecolor='k', lw=1, ms=4, capsize=1.5, alpha=1.0)
        ax2.errorbar(phased_t + 1, resid, fmt='.k', ecolor='k', lw=1, ms=4, capsize=1.5, alpha=1.0)

        ax2.grid()
        ax2.set_xlim(0.0, 2.0)

        ax2.set_xlabel(r'Phase')
        ax2.set_ylabel('Residual')  # \n N$_{terms} = 4$')
    else:
        ax1.errorbar(mjd, mag, err, fmt='.k', ecolor='gray', lw=1, ms=4, capsize=1.5, alpha=1.0)
        if plot_rejected:
            ax1.errorbar(mjd_bad, mag_bad, err_bad, fmt='.r', ecolor='r', lw=1, ms=4, capsize=1.5, alpha=1.0)

        fmagmn = np.mean(mag)
        ferrmn = np.mean(err)
        fmag_stdev = np.std(mag)

        ax1.axhline(fmagmn, color='r', ls='-', lw=2, label='Mean Mag')
        ax1.axhline(fmagmn + 3 * ferrmn, color='g', ls='-.', lw=2, alpha=0.5, label='3X Mag Err')
        ax1.axhline(fmagmn - 3 * ferrmn, color='g', ls='-.', lw=2, alpha=0.5)
        ax1.axhline(fmagmn + 3 * fmag_stdev, color='b', ls=':', lw=2, alpha=0.5, label='3X Mag StDev')
        ax1.axhline(fmagmn - 3 * fmag_stdev, color='b', ls=':', lw=2, alpha=0.5)

        ax1.set_xlabel('MJD')
        ax1.grid()
        ax1.set_ylabel('mag')
        ax1.invert_yaxis()
    return ax1, ax2

<>:206: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:313: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:206: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:313: SyntaxWarning: "is not" with a literal. Did you mean "!="?
/var/folders/pq/0q6qw20n6xxb77g5kk6yy8480000gn/T/ipykernel_49988/1686466588.py:206: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if title is not "":
/var/folders/pq/0q6qw20n6xxb77g5kk6yy8480000gn/T/ipykernel_49988/1686466588.py:313: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if title is not "":
